In [3]:
import math
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)

import yfinance as yf

import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import MinMaxScaler
from torchsummary import summary



In [4]:
from tqdm import tqdm

In [5]:
from transformer_model import TransformerModel
from dataset import Dataset

In [6]:
def calculate_bollinger_bands(data, window=10, num_of_std=2):
    """Calculate Bollinger Bands"""
    rolling_mean = data.rolling(window=window).mean()
    rolling_std = data.rolling(window=window).std()
    upper_band = rolling_mean + (rolling_std * num_of_std)
    lower_band = rolling_mean - (rolling_std * num_of_std)
    return upper_band, lower_band

def calculate_rsi(data, window=10):
    """Calculate Relative Strength Index"""
    delta = data.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=window, min_periods=1).mean()
    avg_loss = loss.rolling(window=window, min_periods=1).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_roc(data, periods=10):
    """Calculate Rate of Change."""
    roc = ((data - data.shift(periods)) / data.shift(periods)) * 100
    return roc

In [7]:
tickers = ['TCS', 'WIPRO', 'HCLTECH', 'INFY', 'LTIM', 'TECHM']

In [8]:
ticker_data_frames = []
stats = {}
for ticker in tickers:
    ticker = ticker + '.NS'
    # Download historical data for the ticker
    data = yf.download(ticker, period="10y", interval="1d")
    # Calculate the daily percentage change
    close = data['Close']
    high = data['High']
    low = data['Low']
    upper, lower = calculate_bollinger_bands(close, window=14, num_of_std=2)
    width = upper - lower
    rsi = calculate_rsi(close, window=14)
    roc = calculate_roc(close, periods=14)
    volume = data['Volume']
    diff = data['Close'].diff(1)
    percent_change_close = data['Close'].pct_change() * 100

    # Create a DataFrame for the current ticker and append it to the list
    ticker_df = pd.DataFrame({
        ticker+'_close': close,
        ticker+'_high': high,
        ticker+'_low': low,
        ticker+'_width': width,
        ticker+'_rsi': rsi,
        ticker+'_roc': roc,
        ticker+'_volume': volume,
        ticker+'_diff': diff,
        ticker+'_percent_change_close': percent_change_close,
    })
    
    MEAN = ticker_df.mean()
    STD = ticker_df.std()

    # Keep track of mean and std
    for column in MEAN.index:
      stats[f"{column}_mean"] = MEAN[column]
      stats[f"{column}_std"] = STD[column]
    
    # # Normalize the training features
    # ticker_df = (ticker_df - MEAN) / STD

    ticker_data_frames.append(ticker_df)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [9]:
# Convert the dictionary containing feature statistics to a DataFrame for easier access
stats = pd.DataFrame([stats], index=[0])

# Display the DataFrame to verify its structure
stats.head()

,TCS.NS_close_mean,TCS.NS_close_std,TCS.NS_high_mean,TCS.NS_high_std,TCS.NS_low_mean,TCS.NS_low_std,TCS.NS_width_mean,TCS.NS_width_std,TCS.NS_rsi_mean,TCS.NS_rsi_std,...,TECHM.NS_rsi_mean,TECHM.NS_rsi_std,TECHM.NS_roc_mean,TECHM.NS_roc_std,TECHM.NS_volume_mean,TECHM.NS_volume_std,TECHM.NS_diff_mean,TECHM.NS_diff_std,TECHM.NS_percent_change_close_mean,TECHM.NS_percent_change_close_std
0,2276.220146,972.562228,2298.934043,980.378417,2253.755093,964.575184,183.042906,114.157928,52.724255,16.496299,...,52.351421,16.870081,0.806511,6.697333,2.922437e+06,2.155744e+06,0.391756,16.08031,0.059683,1.860454


In [10]:
df = pd.concat(ticker_data_frames, axis=1)
df.replace([np.inf, -np.inf], np.nan, inplace=True)
df.dropna(inplace=True)
df.head(2)

,TCS.NS_close,TCS.NS_high,TCS.NS_low,TCS.NS_width,TCS.NS_rsi,TCS.NS_roc,TCS.NS_volume,TCS.NS_diff,TCS.NS_percent_change_close,WIPRO.NS_close,...,LTIM.NS_percent_change_close,TECHM.NS_close,TECHM.NS_high,TECHM.NS_low,TECHM.NS_width,TECHM.NS_rsi,TECHM.NS_roc,TECHM.NS_volume,TECHM.NS_diff,TECHM.NS_percent_change_close
Date,,,,,,,,,,,,,,,,,,,,,
2016-08-10,1337.699951,1341.0,1324.000000,114.252292,78.192144,6.757641,2062078,12.599976,0.950870,203.493805,...,-1.152798,497.200012,504.000000,491.500000,27.128774,44.147423,-1.603003,1326514,-5.299988,-1.054724
2016-08-11,1354.949951,1357.5,1332.775024,102.181835,79.895674,7.745211,2233138,17.250000,1.289527,203.625046,...,-0.898227,500.100006,501.899994,495.299988,26.190954,46.652732,-0.950681,1681953,2.899994,0.583265


* add date end
* add time to sequence

In [11]:
df.shape

(1959, 54)

In [12]:
df[1:11]

,TCS.NS_close,TCS.NS_high,TCS.NS_low,TCS.NS_width,TCS.NS_rsi,TCS.NS_roc,TCS.NS_volume,TCS.NS_diff,TCS.NS_percent_change_close,WIPRO.NS_close,...,LTIM.NS_percent_change_close,TECHM.NS_close,TECHM.NS_high,TECHM.NS_low,TECHM.NS_width,TECHM.NS_rsi,TECHM.NS_roc,TECHM.NS_volume,TECHM.NS_diff,TECHM.NS_percent_change_close
Date,,,,,,,,,,,,,,,,,,,,,
2016-08-11,1354.949951,1357.500000,1332.775024,102.181835,79.895674,7.745211,2233138,17.250000,1.289527,203.625046,...,-0.898227,500.100006,501.899994,495.299988,26.190954,46.652732,-0.950681,1681953,2.899994,0.583265
2016-08-12,1369.224976,1372.400024,1351.025024,102.742022,78.905037,7.014596,2327358,14.275024,1.053546,203.906296,...,0.168120,493.350006,504.000000,488.399994,23.730119,39.959991,-2.960265,3618973,-6.750000,-1.349730
2016-08-16,1345.849976,1370.000000,1336.925049,86.012825,69.820460,5.449340,3183754,-23.375000,-1.707170,201.187546,...,-0.474314,482.950012,491.750000,478.250000,27.134394,37.696684,-3.890545,2087382,-10.399994,-2.108036
2016-08-17,1312.275024,1349.500000,1309.000000,74.355524,55.978129,1.823437,1848566,-33.574951,-2.494702,197.925049,...,-0.439915,474.649994,487.000000,472.200012,33.304215,32.790712,-5.870105,913427,-8.300018,-1.718608
2016-08-18,1319.275024,1328.775024,1315.050049,71.159815,52.640951,0.736854,1292064,7.000000,0.533425,196.912552,...,-0.589145,469.500000,478.700012,468.250000,41.449764,38.923415,-3.305527,1005420,-5.149994,-1.085009
2016-08-19,1301.750000,1323.224976,1300.000000,74.938529,48.026965,-0.603216,2416244,-17.525024,-1.328383,195.037552,...,-1.533452,473.200012,477.200012,470.000000,46.119388,40.000008,-3.022851,3664586,3.700012,0.788075
2016-08-22,1274.349976,1297.224976,1268.150024,94.335486,28.218409,-5.891260,2265274,-27.400024,-2.104861,193.312546,...,-1.286486,466.350006,474.299988,465.000000,53.313767,34.821453,-4.855654,1107412,-6.850006,-1.447592
2016-08-23,1300.525024,1302.474976,1270.099976,95.160224,38.017342,-3.636259,1503374,26.175049,2.053992,194.850052,...,-1.539513,465.799988,474.500000,461.500000,57.696596,26.108738,-6.690710,1617568,-0.550018,-0.117941
2016-08-24,1286.025024,1301.000000,1284.525024,103.516887,39.352961,-3.171705,1116878,-14.500000,-1.114934,194.868805,...,-0.944351,467.049988,471.700012,463.500000,60.301909,28.322457,-6.007246,894108,1.250000,0.268356


In [13]:
df.iloc[11:12]

,TCS.NS_close,TCS.NS_high,TCS.NS_low,TCS.NS_width,TCS.NS_rsi,TCS.NS_roc,TCS.NS_volume,TCS.NS_diff,TCS.NS_percent_change_close,WIPRO.NS_close,...,LTIM.NS_percent_change_close,TECHM.NS_close,TECHM.NS_high,TECHM.NS_low,TECHM.NS_width,TECHM.NS_rsi,TECHM.NS_roc,TECHM.NS_volume,TECHM.NS_diff,TECHM.NS_percent_change_close
Date,,,,,,,,,,,,,,,,,,,,,
2016-08-26,1262.474976,1286.275024,1258.599976,128.379838,35.74789,-4.686503,1250314,-13.050049,-1.023112,183.750046,...,-0.93287,459.950012,463.700012,457.0,66.308905,25.523034,-7.090188,857182,1.850006,0.403843


In [14]:
df.iloc[12]

TCS.NS_close                       1.250275e+03
TCS.NS_high                        1.267475e+03
TCS.NS_low                         1.233375e+03
TCS.NS_width                       1.441112e+02
TCS.NS_rsi                         3.379394e+01
TCS.NS_roc                        -5.621814e+00
TCS.NS_volume                      1.488728e+06
TCS.NS_diff                       -1.219995e+01
TCS.NS_percent_change_close       -9.663519e-01
WIPRO.NS_close                     1.795500e+02
WIPRO.NS_high                      1.843313e+02
WIPRO.NS_low                       1.789125e+02
WIPRO.NS_width                     3.146408e+01
WIPRO.NS_rsi                       8.899314e+00
WIPRO.NS_roc                      -1.278688e+01
WIPRO.NS_volume                    1.358833e+07
WIPRO.NS_diff                     -4.199997e+00
WIPRO.NS_percent_change_close     -2.285712e+00
HCLTECH.NS_close                   3.775500e+02
HCLTECH.NS_high                    3.914750e+02
HCLTECH.NS_low                     3.755

In [15]:
SEQUENCE_LEN = 15  # 15 days of data
BATCH_SIZE = 32
dataset = Dataset(df,SEQUENCE_LEN)
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_set, val_set = torch.utils.data.random_split(dataset, [train_size, test_size])

train_dataloader = torch.utils.data.DataLoader(
            train_set,
            batch_size=BATCH_SIZE,
            shuffle=True,num_workers=4,drop_last=True
)
val_dataloader = torch.utils.data.DataLoader(
            val_set,
            batch_size=BATCH_SIZE,
            shuffle=True,num_workers=4,drop_last=True
)

In [16]:
dataset[0]

(tensor([[ 9.6779e-01,  7.0213e-01,  9.2875e-01,  2.7840e-01,  1.5429e+00,
           1.4299e+00,  1.3520e-01,  8.9498e-01,  8.8310e-01,  1.2236e+00,
           1.3033e+00,  1.1877e+00, -1.1348e+00,  1.4829e+00,  1.3325e+00,
          -8.0408e-01, -6.5730e-01, -5.8231e-01,  1.5585e+00,  1.0352e+00,
           1.3423e+00,  2.0333e+00,  1.6776e+00,  1.6730e+00, -1.3014e+00,
           2.9323e-01,  2.7814e-01,  1.9644e+00,  1.7145e+00,  1.9591e+00,
          -1.6182e+00,  1.7347e+00,  1.9421e+00, -5.8594e-01,  1.5465e-01,
           1.4659e-01,  1.5320e+00,  1.5691e+00,  1.5568e+00, -1.0169e+00,
           1.1935e+00,  1.1668e+00, -8.0716e-01, -6.9881e-01, -6.4552e-01,
           1.6676e+00,  1.6694e+00,  1.6016e+00, -1.2503e+00,  1.2661e+00,
           1.3614e+00, -3.0458e-01, -4.3880e-01, -4.0528e-01],
         [ 1.4251e+00,  1.1675e+00,  1.1673e+00, -1.8054e-01,  1.6361e+00,
           1.6235e+00,  3.9642e-01,  1.1365e+00,  1.1133e+00,  1.2400e+00,
           1.1368e+00,  1.1854e+00, -

In [17]:
def dir_acc(seq,y_true, y_pred):
    y_true_prev = seq[:,-1,0]
    # print(y_true.shape,y_true_prev.shape,y_pred.shape)
    true_change = y_true - y_true_prev  # Calculate true change
    pred_change = y_pred.squeeze(1) - y_true_prev  # Calculate predicted change
    # print(torch.sign(true_change), torch.sign(pred_change))
    correct_direction = torch.eq(torch.sign(true_change), torch.sign(pred_change))  # Check if the signs match
    # print(correct_direction)
    return torch.mean(torch.tensor(correct_direction).float())  # Return the mean of correct directions

In [21]:
# Hyperparameters
input_size = df.shape[1]
output_size = 1
num_layers = 4
d_model = 16
nhead = 4
num_epochs = 100
learning_rate = 0.001
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Instantiate the model, loss function and optimizer
model = TransformerModel(input_size, output_size, d_model, nhead, num_layers)
model.to(device)
criterion = nn.L1Loss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# print(summary(model, (torch.zeros((input_size,15)))))

In [22]:
train_loss_avg = 100
loss_test = 100
avg_dir_accuracy = 0 
# Training loop
for epoch in range(num_epochs):
    
    model.train()
    train_loss_avg = 0
    for seq,target,mean,std in tqdm(train_dataloader):
        optimizer.zero_grad()
        seq = seq.cuda()
        predictions = model(seq)
        # print(predictions.shape)
        loss = criterion(predictions, target.unsqueeze(1).cuda())
        loss.backward()
        optimizer.step()
        train_loss_avg += loss / len(train_dataloader)

    if (epoch + 1) % 1 == 0:
        with torch.no_grad():
            loss_test = 0
            avg_dir_accuracy = 0
            for data in val_dataloader:
                seq,target,mean,std = data
                predictions = model(seq.cuda())
                # validation loss
                batch_loss = criterion(predictions, target.unsqueeze(1).cuda())
                loss_test += batch_loss
                tst = dir_acc(seq.cuda(),target.cuda(),predictions)
                # print('tst',tst)
                avg_dir_accuracy += dir_acc(seq.cuda(),target.cuda(),predictions)
            loss_test /= len(val_dataloader)
            avg_dir_accuracy /= len(val_dataloader)
    print ('-'*15 + f'Epoch:{epoch} Train_loss:{train_loss_avg} Val_loss:{loss_test} Dir Accuracy:f{avg_dir_accuracy}')

  0%|          | 0/48 [00:00<?, ?it/s]

100%|██████████| 48/48 [00:12<00:00,  3.89it/s]
/tmp/ipykernel_1926959/3337551381.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  return torch.mean(torch.tensor(correct_direction).float())  # Return the mean of correct directions


---------------Epoch:0 Train_loss:1.2011609077453613 Val_loss:1.0862655639648438 Dir Accuracy:f0.515625


100%|██████████| 48/48 [00:10<00:00,  4.66it/s]


---------------Epoch:1 Train_loss:1.1395974159240723 Val_loss:1.0818545818328857 Dir Accuracy:f0.5260416865348816


...

---------------Epoch:97 Train_loss:0.44156956672668457 Val_loss:0.7024607062339783 Dir Accuracy:f0.6119791865348816


100%|██████████| 48/48 [00:10<00:00,  4.58it/s]


---------------Epoch:98 Train_loss:0.44737708568573 Val_loss:0.7499611377716064 Dir Accuracy:f0.5807291865348816


100%|██████████| 48/48 [00:10<00:00,  4.67it/s]


---------------Epoch:99 Train_loss:0.4426480531692505 Val_loss:0.7493845820426941 Dir Accuracy:f0.5703125
